<center><img src="../logo.png" alt="Header" style="width: 800px;"/></center>


@Copyright (C): 2010-2020, Shenzhen Yahboom Tech  
@Author: Liusen  
@Date: 2020-02-24 11:10:02  
@LastEditors: Liusen  
@LastEditTime: 2020-02-24 11:10:02    

In [ ]:
import enum
import cv2

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

### Define gesture recognition function

In [ ]:
import cv2 
import time
import demjson
import pygame 
from aip import AipBodyAnalysis
from aip import AipSpeech
from PIL import Image, ImageDraw, ImageFont
import numpy
import ipywidgets.widgets as widgets

hand={'One':'number_1','Five':'number_5','Fist':'fist','Ok':'OK',
      'Prayer':'prayer','Congratulation':'congratulation','Honour':'honour',
      'Heart_single':'比心心','Thumb_up':'点赞','Thumb_down':'Diss',
      'ILY':'i love you','Palm_up':'palm_up','Heart_1':'heart_1',
      'Heart_2':'heart_2','Heart_3':'heart_3','Two':'number_2',
      'Three':'number_3','Four':'number_4','Six':'number_6','Seven':'number_7',
      'Eight':'number_8','Nine':'number_9','Rock':'Rock','Face':'face'}

""" APPID AK SK """
APP_ID = '24840713'
API_KEY = 'WMvYkrI7omD8OyGlwqjHQMRK'
SECRET_KEY = 'XAKxY0NyCL1QIfIRjEi8ScsPjGjsANuX'

#camera = PiCamera()
client = AipBodyAnalysis(APP_ID, API_KEY, SECRET_KEY)

g_camera = cv2.VideoCapture(0)
g_camera.set(3, 640)
g_camera.set(4, 480)
g_camera.set(5, 30)  #set frame
g_camera.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
g_camera.set(cv2.CAP_PROP_BRIGHTNESS, 60) #设置亮度 -64 - 64  0.0
g_camera.set(cv2.CAP_PROP_CONTRAST, 50) #设置对比度 -64 - 64  2.0
g_camera.set(cv2.CAP_PROP_EXPOSURE, 156) #设置曝光值 1.0 - 5000  156.0

ret, frame = g_camera.read()

# Define camera display components

In [ ]:
image_widget = widgets.Image(format='jpeg', width=640, height=480)  
display(image_widget)   
image_widget.value = bgr8_to_jpeg(frame)

# Define conversion display Chinese function

In [ ]:
def cv2ImgAddText(img, text, left, top, textColor=(0, 255, 0), textSize=20):
    if (isinstance(img, numpy.ndarray)):  # 判断是否OpenCV图片类型
        img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    draw = ImageDraw.Draw(img)

    fontStyle = ImageFont.truetype(
        "simhei.ttf", textSize, encoding="utf-8")

    draw.text((left, top), text, textColor, font=fontStyle)

    return cv2.cvtColor(numpy.asarray(img), cv2.COLOR_RGB2BGR)

# Main process

In [ ]:
while True:
    """Take a pciture """
    ret, frame = g_camera.read()
    #retval, frame =   g_camera.read()
    """ call gesture """
    raw = str(client.gesture(image_widget.value))
    text = demjson.decode(raw)
    try:
        res = text['result'][0]['classname']
    except:
        #print('nothing' )
#       cv2.putText(frame, 'unrecognized', (250,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,200), 2, cv2.LINE_AA) #只能显示英文
        img = cv2ImgAddText(frame, "unrecognized", 250, 30, (0, 0 , 255), 30)
    else:
        print('Recognition result:' + hand[res])
        #cv2.putText(frame, hand[res], (250,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2, cv2.LINE_AA)
        img = cv2ImgAddText(frame, hand[res], 250, 30, (0, 255 , 0), 30)

    image_widget.value = bgr8_to_jpeg(img)
          

In [ ]:
g_camera.release()#End the process and release the camera